In [ ]:
# %load assign3.py
#https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
import csv
import numpy as np
from string import maketrans
import string
trainlist=[]
testlist=[]
testlistACGT=[]
X = []
Y = []
XForTest = []
Yfortest = []
#DNAdict = {'A':1,'C':2,'G':3,'T':4}
with open('train.csv','rb') as traincsvFile:
    lines=csv.reader(traincsvFile)
    for line in lines:
        trainlist.append(list(line))
        
    firstrow = trainlist[0]
    del trainlist[0] #delete first row of train.csv
    
    #print trainlist
    #trainlistrow = [x[1] for x in trainlist]
    translatetable = maketrans('ACGT', '1234')
    #print trainlist
    for eachrow in trainlist:
        eachrow[0] = string.atoi(eachrow[0])
        # print 'eachrow=' + eachrow[1]
        eachrow[1] = eachrow[1].translate(translatetable)
        #print 'new_eachrow=' + eachrow[1]
        Xstring = '-'.join(eachrow[1]).split('-')
        Xstring = map(eval, Xstring)
        #print Xstring
        X.append(Xstring)
        eachrow[2] = string.atoi(eachrow[2])
        Y.append(eachrow[2])
    #print trainlist
    #print Y
    #print X

    

### begin train model

In [ ]:
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation 
    import numpy
    from keras.optimizers import SGD  
    from keras.layers import LSTM
    from keras import regularizers
    from keras.wrappers.scikit_learn import KerasClassifier
    from sklearn.model_selection import GridSearchCV


    # fix random seed for reproducibility
    seed = 7
    numpy.random.seed(seed)

    Xinput = np.array(X)
    Yinput = np.array(Y)
    
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=14, init='uniform', activation='relu'))
#     model.add(Dropout(0.5)) 
    model.add(Dense(8, init='uniform', activation='relu'))
#     model.add(Dropout(0.5)) 
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)  

    # Compile model
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer='Adagrad', metrics=['accuracy'])


    # Fit the model
    model.fit(Xinput, Yinput, epochs=600, batch_size=10, verbose=2)
    # calculate predictions
    # predictions = model.predict(np.array(X))
    # # round predictions
    # rounded = [round(x[0]) for x in predictions]
    # print(rounded)
# 4. 评估模型
    loss, accuracy = model.evaluate(np.array(X), np.array(Y))
    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 5. 数据预测
    probabilities = model.predict(np.array(X))
    predictions111 = [float(round(x)) for x in probabilities]
    accuracy = numpy.mean(predictions111 == Y)
    print("Prediction Accuracy: %.2f%%" % (accuracy*100))


### begin test predictions

In [ ]:
    with open('test.csv', 'rb') as testcsvFile:
        lines = csv.reader(testcsvFile)
        for line in lines:
            testlist.append(list(line))
            testlistACGT.append(list(line))
        firstrow = testlist[0]
        del testlist[0]  # delete first row of train.csv
        del testlistACGT[0]
        
        # trainlistrow = [x[1] for x in trainlist]
        translatetable = maketrans('ACGT', '1234')
#         print testlist
        for eachrow in testlist:
            #eachrow[0] = string.atoi(eachrow[0])
            # print 'eachrow=' + eachrow[1]
            eachrow[1] = eachrow[1].translate(translatetable)
            # print 'new_eachrow=' + eachrow[1]
            Xstringtest = '-'.join(eachrow[1]).split('-')
            Xstringtest = map(eval, Xstringtest)
            # print Xstring
            XForTest.append(Xstringtest)
            # eachrow[2] = string.atoi(eachrow[2])
            # Yfortest.append(eachrow[2])
        #print testlist
        #print XForTest
        # calculate predictions
        predictions = model.predict(np.array(XForTest))
        # round predictions
        rounded = [int(round(x[0])) for x in predictions]
#         print(rounded)

In [ ]:
print type(rounded)


### output to csv file

In [ ]:
idofdna=[x[0] for x in testlistACGT]
# print idofdna #useful！！！
# print testlistACGT
outputlist = np.c_[np.array(idofdna),np.array(rounded)].tolist()
# print outputlist

In [ ]:
print type(outputlist)
firstrowtest = ['id','prediction']
# numpy.savetxt('outputtest.csv', outputlist, delimiter = ',',fmt="%f,%f,%f") 
with open("outputtest.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerow(firstrowtest)
    writer.writerows(outputlist)

In [ ]:
print type(outputlist)
print firstrowtest

### fine-tune on epochs and batch_size

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=14, activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [100, 500, 1000]
param_grid = dict(batch_size=batch_size, nb_epoch=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


### fine-tune on optimizer

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(optimizer='adam'):
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=14,init='uniform', activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform',activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)

# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=600, batch_size=10, verbose=2)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### fine tune on learning rate and momentum

In [2]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
# Function to create model, required for KerasClassifier
# create model
def create_model(learn_rate=0.01, momentum=0):
    model = Sequential()
    model.add(Dense(12, input_dim=14, activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)

# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=10, verbose=2)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


/Users/haoranyu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
/Users/

Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
 - 1s - loss: 0.6937 - acc: 0.4951
 - 1s - loss: 0.5993 - acc: 0.7502
 - 2s - loss: 0.6513 - acc: 0.6747
 - 1s - loss: 0.5944 - acc: 0.7502
Epoch 1/1
Epoch 1/1
 - 1s - loss: 0.6937 - acc: 0.5379
 - 1s - loss: 0.6084 - acc: 0.7504
Epoch 1/1
Epoch 1/1
 - 1s - loss: 0.6412 - acc: 0.6984
 - 1s - loss: 0.6936 - acc: 0.5379
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
 - 1s - loss: 0.6253 - acc: 0.7256
 - 1s - loss: 0.5792 - acc: 0.7502
 - 1s - loss: 0.6901 - acc: 0.5199
 - 1s - loss: 0.6110 - acc: 0.7301
Epoch 1/1
Epoch 1/1
Epoch 1/1
Epoch 1/1
 - 1s - loss: 0.6093 - acc: 0.7502
 - 1s - loss: 0.6900 - acc: 0.5146
 - 2s - loss: 0.6756 - acc: 0.7519
 - 1s - loss: 0.5973 - acc: 0.7502
Epoch 1/1
Epoch 1/1
Epoch 1/1
 - 2s - loss: 0.6924 - acc: 0.5146
Epoch 1/1
 - 2s - loss: 0.5790 - acc: 0.7502
 - 2s - loss: 0.6171 - acc: 0.7084
 - 2s - loss: 0.6926 - acc: 0.5154
Epoch 1/1
Epoch 1/1
Epoch 1/1
 - 2s - loss: 0.5884 - acc: 0.7361
 - 2s - loss: 0.5703 - acc: 0.7502


/Users/haoranyu/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]


Epoch 1/1
 - 1s - loss: 0.6977 - acc: 0.4920
Best: 0.206000 using {'learn_rate': 0.1, 'momentum': 0.8}
0.182000 (0.257290) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.183500 (0.259411) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.185500 (0.262238) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.181000 (0.255877) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.179500 (0.253756) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.169500 (0.239619) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.174500 (0.246688) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.179500 (0.253756) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.187000 (0.264359) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.180000 (0.254463) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.178000 (0.251636) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.188000 (0.265772) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.167000 (0.236085) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.191000 (0.270014) with: {'learn_rate': 0.1, 'momentum': 0.

### fine tune on init_mode

In [ ]:
# Use scikit-learn to grid search the weight initialization
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
# create model
def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=14, activation='relu'))
    model.add(Dense(8, init=init_mode, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)


# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=10, verbose=2)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### fine tune on activation function

In [ ]:
# Use scikit-learn to grid search the activation function
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model(activation='relu'):
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=14, activation=activation))
    model.add(Dense(8, init='uniform', activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)


# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=10, verbose=2)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### fine tune on dropout_rate and weight_constraint

In [ ]:
# Use scikit-learn to grid search the dropout rate
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(dropout_rate=0.0, weight_constraint=0):
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=14, activation='relu'))
    model.add(Dropout(dropout_rate))

    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)

# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=10, verbose=2)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


### fine tune on num of neurons

In [ ]:
# Use scikit-learn to grid search the number of neurons
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
# Function to create model, required for KerasClassifier
def create_model(neurons=1):
# create model
    model = Sequential()
    model.add(Dense(neurons, input_dim=14,init='uniform' ,activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
Xinput = np.array(X)
Yinput = np.array(Y)

# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=300, batch_size=10, verbose=2)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(Xinput, Yinput)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

## 